![alt text](files/roofitbasics.png "Title")

<b>RooFit</b> is a OO analysis environment built on ROOT. It is essentially a collection of classes designed to augment root for data modeling whose aim is summarised below (shamelessly stolen from Wouter Verkerke)...

![alt text](files/roofit.png)


We will cover a few of the basics in the session today but note there are many more tutorials available [here](https://root.cern.ch/root/html600/tutorials/roofit/index.html)

In [ ]:
#Setup python in the usual way 
import ROOT
import os
%jsroot on

## RooFit objects

In Roofit, any variable, data point, function, PDF ... is represented by a c++ object

The most basic of these is the `RooRealVar`. Let's create one which will represent the mass of some hypothetical particle, we name it and give it an initial starting value and range.

In [ ]:
MH = ROOT.RooRealVar("MH","mass of the Hypothetical Boson (H-boson) in GeV",125,100,150)
MH.Print()

ok, great. Now we can access this object and modify it. 

In [ ]:
MH.setVal(130)
MH.getVal()

In CMS we don't observe this particle mass but usually define some observable which is *sensitive* to this mass. Lets assume we reconstruct the decay products of the H-boson and measure the invariant mass of those particles. We need to make another variable which represents that invariant mass

In [ ]:
mass = ROOT.RooRealVar("mass","Invariant mass of our decay products in GeV",100,80,200)
mass.Print()

In the perfect world we would perfectly measure the pole mass of the particle in every single event (assuming there is no intrinsic width of the particle). However, CMS is far from perfect so there will be some resolution effect. Lets assume the resolution of our measurement of the invariant mass is 10 GeV and call it "sigma"

In [ ]:
sigma = ROOT.RooRealVar("sigma","Invariant mass resolution in GeV",10,0,20)
sigma.Print()

More exotic variables can be constructed out of these `RooRealVar`s using `RooFormulaVars`. For example, suppose we wanted to make a function out of the variables which represented the relative resolution as a function of the hypothetical mass MH. 

In [ ]:
function = ROOT.RooFormulaVar("rel_resolution","@0/@1",ROOT.RooArgList(sigma,mass))
function.Print("v")

Notice how there is a list of the variables we passed (the servers or "actual vars"). We can now plot the function ... 

In [ ]:
plot = mass.frame()
function.plotOn(plot)

c = ROOT.TCanvas()
plot.Draw()
c.Draw()

The main objects we are interested in using from RooFit are <b>"probability denisty functions"</b> or (PDFs). We can construct the PDF `f(mass|MH,sigma)` as a simple Gaussian shape for example or a `RooGaussian` in RooFit language (think McDonald's logic, everything is a `RooSomethingOrOther`)

In [ ]:
gauss = ROOT.RooGaussian("gauss","Gaussian PDF",mass,MH,sigma)
gauss.Print("V")

Notice how the gaussian PDF, like the `RooFormulaVar` depends on our `RooRealVar` objects, these are its servers.  Its evaluation will depend on their values. 

The main difference between PDFs and Functions in RooFit is that PDFs are automatically normalised to unitiy, hence they represent a probability density, you don't need to normalise yourself. Lets plot it for the current values

In [ ]:
plot = mass.frame()
c0 = ROOT.TCanvas()
gauss.plotOn(plot)
plot.Draw()
c0.Draw()

If we change the value of MH, the PDF gets updated too!

In [ ]:
MH.setVal(125)
gauss.plotOn(plot,ROOT.RooFit.LineColor(ROOT.kRed))

MH.setVal(135)
gauss.plotOn(plot,ROOT.RooFit.LineColor(ROOT.kGreen))

plot.Draw()
c0.Update()
c0.Draw()

We can generate MC data from a PDF in RooFit using a single line! Note that we have to tell RooFit which variables to generate in. In this case, each of our events will be a single value of "mass"

In [ ]:
data = gauss.generate(ROOT.RooArgSet(mass),ROOT.RooFit.NumEvents(500))
data.Print()

In [ ]:
plot2 = mass.frame()
data.plotOn(plot2)
gauss.plotOn(plot2)
gauss.paramOn(plot2)
c1 = ROOT.TCanvas()
plot2.Draw()
c1.Draw()

Of course at CMS, we're not in the business of generating MC events, but collecting <b>real data!</b>. Lets see how we can use our real data in RooFit.

## RooFit datasets

A dataset is essentially just a collection of points in N-dimensional (N-observables) space. There are two basic implementations in RooFit, 

1) an "unbinned" dataset - `RooDataSet`

2) a "binned" dataset - `RooDataHist`

both of these use the same basic structure as below

![alt text](files/datastructure.png)

Lets create an empty dataset where the only observable, the mass. Points can be added to the dataset one by one ...

In [ ]:
mydata = ROOT.RooDataSet("dummy","My dummy dataset",ROOT.RooArgSet(mass))

mass.setVal(123.4)
mydata.add(ROOT.RooArgSet(mass))
mass.setVal(145.2)
mydata.add(ROOT.RooArgSet(mass))
mass.setVal(170.8)
mydata.add(ROOT.RooArgSet(mass))


mydata.Print()
print "YAWN!"

There are also other ways to manipulate datasets in this way as shown in the diagram below 

![alt text](files/datasets_manip.png)


Luckily there are also Constructors for a `RooDataSet` from a `TTree` and for a `RooDataHist` from a `TH1` so its simple to convert from your usual ROOT objects.

Let's take an example dataset put together already.

In [ ]:
fi = ROOT.TFile.Open("tutorial.root")
fi.ls()

Inside the file, there is something called a `RooWorkspace`. This is just the RooFit way of keeping a persistent link between the objects for a model. It is a very useful way to share data and PDFs/functions etc among CMS collaborators.

Let's take a look at it. It contains a `RooDataSet` and one variable. This time we called our variable (or observable) `CMS_hgg_mass`, let's assume now that this is the invariant mass of photon pairs where we assume our H-boson decays to photons.  

In [ ]:
wspace = fi.Get("workspace")
wspace.Print("v")

Let's have a look at the data. The `RooWorkspace` has several accessor functions, we will use the `RooWorkspace::data` one, note there is als `RooWorkspace::var`, `RooWorkspace::function` and `RooWorkspace::pdf` with (hopefully) obvious purposes.

In [ ]:
data = wspace.data("dataset")
mass = wspace.var("CMS_hgg_mass")
plot = mass.frame()

data.plotOn(plot,ROOT.RooFit.Binning(160))

c2 = ROOT.TCanvas()
plot.Draw()
c2.Draw()

## Fitting to data 

Let's try to fit this data with a pdf. Since it looks like a falling distribution, lets try an exponential. 

RooFit can fit PDFs to the data using an unbinned maximum likelihood approach. It even has a very simple interface, `RooAbsPdf::fitTo`. RooFit will figure out the range to fit to and also that it should fit the parameter <b>alpha</b> since that variable isn't in our dataset. 

In [ ]:
alpha = ROOT.RooRealVar("alpha","slope parameter for exponential",-0.5,-2,0.01)
exp = ROOT.RooExponential("exp","exponential function",mass,alpha)
exp.fitTo(data)

exp.plotOn(plot)
exp.paramOn(plot)
plot.Draw()
c2.Update()
c2.Draw()

RooFit has found the best fit value of alpha for this dataset. It also estimates an uncertainty on alpha using the Hessian matrix from the fit.

<b><span style="color:red">Q: Can you figure out how to pass options to the `fitTo` method to change the range for fitting?.</span></b> 


<b><span style="color:red">Q: What other functions can you think of which would also fit this data? Try a different function.</span></b> 

<b>Hint:</b> You can fund a bunch of available RooFit functions [here](https://root.cern.ch/root/html/ROOFIT_ROOFIT_Index.html), there is also support for a generic pdf in the form of a `RooGenericPdf`, check [this link](https://root.cern.ch/doc/v608/classRooGenericPdf.html)

It looks like there could be a small region near 125 GeV for which our fit doesn't quite go through the points. Maybe our hypothetical H-boson isn't so hypothetical after all!

Let's see what happens if we include some resonant signal into the fit. We can take our Gaussian function again and use that as a signal model. A reasonable value for the resolution of a resonant signal with a mass around 125 GeV decaying to a pair of photons is around a GeV.

In [ ]:
sigma.setVal(1.)
sigma.setConstant()

MH.setVal(125)
MH.setConstant()

signal = ROOT.RooGaussian("signal","Gaussian PDF",mass,MH,sigma)

We need to add this to our exponential model and fit a "Sigmal+Background model". In RooFit there are two ways to add PDFs, recursively where the fraction of yields for the signal and background is a parameter or absolutely where each PDF has its own normalisation. We're going  to use the second one.

In [ ]:
norm_s = ROOT.RooRealVar("norm_s","Number of signal events",0,500)
norm_b = ROOT.RooRealVar("norm_b","Number of background events",0,1000)

model = ROOT.RooAddPdf("model","Sigma + Background PDF",ROOT.RooArgList(signal,exp),ROOT.RooArgList(norm_s,norm_b))
model.Print("v")

We can import this new model into our `RooWorkspace` and show off our new discovery to our CMS friends (if we weren't about 4.5 years too late!)

In [ ]:
getattr(wspace,"import")(model)  # this is needed since "import" is a keyword in python
wspace.Print("v")

Ok now lets fit the model. Note this time we add the option "`Extended()`" which tells RooFit that we care about the overall number of events too. It will add an additional Poisson term in the likelihood to account for this.  

In [ ]:
model.fitTo(data,ROOT.RooFit.Extended())

In [ ]:
plot = mass.frame()
data.plotOn(plot)

model.plotOn(plot,ROOT.RooFit.Components("exp"),ROOT.RooFit.LineColor(ROOT.kRed))
model.plotOn(plot)

model.paramOn(plot)

c4 = ROOT.TCanvas()
plot.Draw()
c4.Draw()

What about if we also fit for the mass (MH)? we can easily do this by removing the constant setting on MH...

In [ ]:
MH.setConstant(False)
model.fitTo(data,ROOT.RooFit.Extended())

Notice now the result for the fitted MH is not 125 and gets added to the fitted parameters since now it is floating.

<span style="color:red"><b>Q: Compare the uncertainties on r with and without the freely floating MH. Should they be different?</b>

We can get more information about the fit using the option "`Save()`". 

In [ ]:
fit_res = model.fitTo(data,ROOT.RooFit.Extended(),ROOT.RooFit.Save())
fit_res.Print("v")

For example, we can get the Correlation Matrix from the fit result... Note that the order of the parameters are the same as listed in the "Floating Parameter" list above

In [ ]:
cormat = fit_res.correlationMatrix()
cormat.Print()

And even some fancy ways to visualise these results ... 

In [ ]:
c = ROOT.TCanvas("c","c",1000,460)
c.Divide(2)

c.cd(1)
plot_err_mat = ROOT.RooPlot(norm_b,norm_s,900,1100,0,80)
fit_res.plotOn(plot_err_mat,norm_b,norm_s,"MEVH12")
plot_err_mat.Draw()

c.cd(2)
model.plotOn(plot,ROOT.RooFit.VisualizeError(fit_res,2),ROOT.RooFit.FillColor(ROOT.kOrange))
model.plotOn(plot)
plot.Draw()

c.Draw()

<span style="color:red"><b>Q: We could also determine the uncertainties from toys. Without going full Feldman-Cousins, throw toys and determine an uncertainty on MH and norm_s and their correlation from toys.</b>

Hint: You could use the [RooMCStudy](https://root.cern.ch/root/html600/tutorials/roofit/rf801_mcstudy.C.html) class for this

In [ ]:
# Run the toy studies